## 设计并训练KNN算法对图片进行分类

In [1]:
import tensorflow as tf  
import numpy as np  

In [2]:
'''
def loadMNIST():  
    from tensorflow.examples.tutorials.mnist import input_data  
    mnist = input_data.read_data_sets('data/fashion',dtype='uint8')  
    return mnist
'''

"\ndef loadMNIST():  \n    from tensorflow.examples.tutorials.mnist import input_data  \n    mnist = input_data.read_data_sets('data/fashion',dtype='uint8')  \n    return mnist\n"

In [3]:
'''
def nomalizing(array):
    m,n = np.shape(array)#shape函数为得到数组的各个维度
    for i in xrange(m):
        for j in xrange(n):
            if array[i,j] != 0:
                array[i,j] = 1
    return array
'''


#### 定义读取TFRecord格式数据的函数

In [15]:
def loadtraindata(trainfile, tensor, num):
    filename_queue = tf.train.string_input_producer([trainfile])  #引入tfrecord文件
    #读取
    reader = tf.TFRecordReader()  
    _, example = reader.read(filename_queue)  
    features = tf.parse_single_example(example,  
                                       features={'lab': tf.FixedLenFeature([], tf.int64),  
                                                 'img': tf.FixedLenFeature(tensor, tf.float32)})  
    image = tf.cast(features['img'], tf.float64)  
   # image = tf.reshape(image, [28, 28]) 
    label = tf.cast(features['lab'], tf.int32) 

    #创建序列，要保存文件内容
    images = list()
    labels = list()
    
    #转换文件格式
    with tf.Session() as sess:  
        init = tf.global_variables_initializer()  
        sess.run(init)  
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)  
        for i in range(num):  
            img, lab = sess.run([image, label])  
            images.append(np.array(img[0]))
            tem=np.zeros((1,10))
            tem[0][lab]=1.0
            labels.append(tem[0])
            del tem
            
        coord.request_stop()  
        coord.join(threads)     
   
    #保存为数组格式
    images=np.array(images)
    labels=np.array(labels)
    
    return images,labels


#### 定义KNN算法分类数据

In [18]:
def KNN():  
    #读取训练集和测试集数据
    train_x,train_y=loadtraindata(trainfile='train.tfrecords',tensor=[1,784],num=60000)
    test_x, test_y =loadtraindata(trainfile='test.tfrecords',tensor=[1,784],num=5000) 
    #定义参数  
    xtr = tf.placeholder(tf.float32,[None,784])  
    xte = tf.placeholder(tf.float32,[784])  
    
    #计算距离 
    #运用欧式距离公式：d=sqrt( ∑(xi1-xi2)^ ) i=1,2..n 
    distance = tf.sqrt(tf.reduce_sum(tf.pow(tf.add(xtr, tf.negative(xte)),2),reduction_indices=1))  
      
    pred = tf.argmin(distance,0)  
      
    init = tf.initialize_all_variables()  
      
    sess = tf.Session()  
    sess.run(init)  
      
    right = 0  
    #测试准确度
    for i in range(200):  
        ansIndex = sess.run(pred,{xtr:train_x,xte:test_x[i,:]})  
        #print ('prediction is ',np.argmax(train_y[ansIndex])  )
        #print ('true value is ',np.argmax(test_y[i])  )
        if np.argmax(test_y[i]) == np.argmax(train_y[ansIndex]):  
            right += 1.0  
    accracy = right/200.0  
    print ('精确度:' , accracy)

In [19]:
if __name__ == "__main__":  
     
    KNN()  

Instructions for updating:
Use `tf.global_variables_initializer` instead.
精确度: 1.0
